<a href="https://colab.research.google.com/github/PhoenixStormJr/Unfinished-Colab-AI-enhance-video/blob/main/Enhance_Video_With_Only_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title establish session, upload video, get .wav, framerate, and split video into frames
#@markdown if you would like to start over, delete the folder "UpscaleVideo" in your google drive.
#@markdown
#@markdown Make sure before running the master session, UpscaleVideo/MergeSessions is deleted
%cd /content
#@markdown Mounting google drive also automatically combines sessions. Use multiple sessions to super speed the time!
MountGoogleDrive = True # @param {type:"boolean"}
#@markdown In case you're having errors or glitches, you can force this to be the master session by checking this box
ForceMasterSession = False # @param {type:"boolean"}

format = "png" # @param ["png", "jpg", "tga", "tiff", "webp"]

MergeSessionsPath = "/content/drive/MyDrive/UpscaleVideo/MergeSessions"
OriginalOutputPath = "/content/OriginalOutput"
UpscaledOutputPath = "/content/drive/MyDrive/UpscaleVideo/UpscaledOutput"



def main():
  import os
  global session
  session = 0
  if(os.path.exists("/content/sessionNumber.txt")):
    session = int(open("/content/sessionNumber.txt","r").read())
    print("Session already established.")
    if(session == 1):
      print("This is the master session.")
    else:
      print(f"This is an assist session {session}.")
  if((MountGoogleDrive) and (not os.path.exists("/content/drive/MyDrive"))):
    from google.colab import drive
    drive.mount('/content/drive')
    if(not os.path.exists(f"{MergeSessionsPath}/setup")):
      os.makedirs(f"{MergeSessionsPath}/setup")
    if(not os.path.exists("/content/sessionNumber.txt")):
      if(ForceMasterSession):
        !rm -r {MergeSessionsPath}/setup
        os.makedirs(f"{MergeSessionsPath}/setup")
        !rm -r {MergeSessionsPath}/ToProcess
      if((not os.path.exists(f"{MergeSessionsPath}/MasterSessionTaken.txt")) or (ForceMasterSession)):
        print()
        print()
        print("MergeSessionsPath/setup contains this:")
        print()
        print(os.listdir(f"{MergeSessionsPath}"))
        f = open(f"{MergeSessionsPath}/MasterSessionTaken.txt", "w")
        f.close()
        session = 1
        open("/content/sessionNumber.txt","w").write(str(session))
        print(f"SESSION IS: {session}, AKA the master session.")
        print()
        print()
        print("After establishing session, MergeSessionsPath contains this:")
        print()
        print(os.listdir(f"{MergeSessionsPath}"))
      else:
        x = 2
        while(os.path.exists(f"{MergeSessionsPath}/setup/Session{x}Taken.txt")):
          x = x + 1
        print()
        print()
        print("MergeSessionsPath/setup contains this:")
        print()
        print(os.listdir(f"{MergeSessionsPath}/setup"))
        f = open(f"{MergeSessionsPath}/setup/Session{x}Taken.txt", "w")
        f.close()
        session = x
        open("/content/sessionNumber.txt","w").write(str(session))
        print(f"SESSION IS {session} AKA an assist session.")
        print()
        print()
        print("After establishing session, MergeSessionsPath/setup contains this:")
        print()
        print(os.listdir(f"{MergeSessionsPath}/setup"))
  from google.colab import files
  import os

  if(not os.path.exists("/content/drive/MyDrive/UpscaleVideo")):
    os.makedirs("/content/drive/MyDrive/UpscaleVideo")
  if(not os.path.exists("/content/drive/MyDrive/UpscaleVideo/OriginalOutput.zip")):
    if(not os.path.exists(OriginalOutputPath)):
      os.makedirs(OriginalOutputPath)
    uploaded = files.upload()
    filename = list(uploaded)[0]
    !rm /content/input.mp4
    os.rename(filename, "/content/input.mp4")
    filename2 = filename.replace(" ","_").replace("(","_").replace(")","_")
    print(filename2)
    !ffmpeg -i input.mp4 /content/drive/MyDrive/UpscaleVideo/audio.wav
    FrameRate1 = !ffmpeg -i "/content/input.mp4" 2>&1 | sed -n "s/.*, \(.*\) tbr.*/\1/p"
    print(FrameRate1)
    for thing in FrameRate1:
      FrameRate = int(thing)
    open("/content/drive/MyDrive/UpscaleVideo/Framerate.txt", "w").write(str(FrameRate))
    #I have to zip it, because google drive only shares a few files at a time.
    !ffmpeg -i input.mp4 {OriginalOutputPath}/%07d.{format}
    !zip -r /content/drive/MyDrive/UpscaleVideo/OriginalOutput.zip {OriginalOutputPath}
  elif(not os.path.exists(OriginalOutputPath)):
    !cp /content/drive/MyDrive/UpscaleVideo/OriginalOutput.zip /OriginalOutput.zip
    %cd /
    !unzip OriginalOutput.zip
    %cd /content
  if(session == 1):
    print(f"Session is {session}, AKA the MASTER SESSION.")
  else:
    print(f"Session is {session}, AKA an assist session.")

try:
  main()
except KeyboardInterrupt:
  print('Interrupted')
  session = int(open("/content/sessionNumber.txt","r").read())
  if(session > 1):
    !rm {MergeSessionsPath}/setup/Session{session}Taken.txt
    raise Exception("KeyboardInterrupt: You have terminated this colab notebook. Sending message to main notebook it is closed.")
  else:
    !rm {MergeSessionsPath}/MasterSessionTaken.txt
    !rm -r {MergeSessionsPath}/setup
    raise Exception("KeyboardInterrupt: You have terminated the main colab notebook. Terminating all others...")

In [ ]:
#@title Setup only. For ESRGAN
# @markdown This is extremely long setup code. I don't reccomend viewing it unless you're a geek, and like that stuff.

MergeSessionsPath = "/content/drive/MyDrive/UpscaleVideo/MergeSessions"
OriginalOutputPath = "/content/OriginalOutput"
UpscaledOutputPath = "/content/drive/MyDrive/UpscaleVideo/UpscaledOutput"


def main():
  import os
  #changed install to fix error.
  #https://www.patreon.com/posts/for-people-error-100361567
  if(not os.path.exists("/usr/local/lib/python3.10/dist-packages/torchaudio-2.0.2+cu118.dist-info")):
    !pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
  if(not os.path.exists("/usr/local/lib/python3.10/dist-packages/xformers-0.0.21.dist-info")):
    !pip install xformers==0.0.21
    !pip install basicsr
  if(not os.path.exists("/usr/local/lib/python3.10/dist-packages/gfpgan")):
    !pip install gfpgan
#@title Now run this next cell to import all necessary packages:
def kill():
  print('Interrupted')
  session = int(open("/content/sessionNumber.txt","r").read())
  if(session > 1):
    !rm {MergeSessionsPath}/setup/Session{session}Taken.txt
    raise Exception("KeyboardInterrupt: You have terminated this colab notebook. Sending message to main notebook it is closed.")
  else:
    !rm {MergeSessionsPath}/MasterSessionTaken.txt
    !rm -r {MergeSessionsPath}/setup
    raise Exception("KeyboardInterrupt: You have terminated the main colab notebook. Terminating all others...")

try:
  main()
except KeyboardInterrupt:
  kill()
except OSError:
  kill()
except ModuleNotFoundError:
  kill()



import cv2
import math
import numpy as np
import os
import queue
import threading
import torch
from torch.nn import functional as F
from torch import nn as nn
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from math import ceil, floor, sqrt
from PIL import Image, ImageFilter
from IPython.display import Image as display_image





#The rest of this stuff is just functions. Don't worry about it.
#@title Utilities
#These are the Utils functions and the classes containing the actual model architecture:
# Utils
# ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

def convert_to_jpg(input_path, output_path):
    # Open the PNG file
    with Image.open(input_path) as img:
        # Save the image in JPEG format
        img.convert("RGB").save(output_path, 'JPEG')


class RealESRGANer():
    """A helper class for upsampling images with RealESRGAN.

    Args:
        scale (int): Upsampling scale factor used in the networks. It is usually 2 or 4.
        model_path (str): The path to the pretrained model. It can be urls (will first download it automatically).
        model (nn.Module): The defined network. Default: None.
        tile (int): As too large images result in the out of GPU memory issue, so this tile option will first crop
            input images into tiles, and then process each of them. Finally, they will be merged into one image.
            0 denotes for do not use tile. Default: 0.
        tile_pad (int): The pad size for each tile, to remove border artifacts. Default: 10.
        pre_pad (int): Pad the input images to avoid border artifacts. Default: 10.
        half (float): Whether to use half precision during inference. Default: False.
    """

    def __init__(self,
                 scale,
                 model_path,
                 model=None,
                 tile=0,
                 tile_pad=10,
                 pre_pad=10,
                 half=False,
                 device=None,
                 gpu_id=None):
        self.scale = scale
        self.tile_size = tile
        self.tile_pad = tile_pad
        self.pre_pad = pre_pad
        self.mod_scale = None
        self.half = half

        # initialize model
        if gpu_id:
            self.device = torch.device(
                f'cuda:{gpu_id}' if torch.cuda.is_available() else 'cpu') if device is None else device
        else:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') if device is None else device
        # if the model_path starts with https, it will first download models to the folder: realesrgan/weights
        # if model_path.startswith('https://'):
        #     model_path = load_file_from_url(
        #         url=model_path, model_dir='realesrgan/weights')
        loadnet = torch.load(model_path, map_location=torch.device('cpu'))
        # prefer to use params_ema
        if 'params_ema' in loadnet:
            keyname = 'params_ema'
        else:
            keyname = 'params'
        model.load_state_dict(loadnet[keyname], strict=True)
        model.eval()
        self.model = model.to(self.device)
        if self.half:
            self.model = self.model.half()

    def pre_process(self, img):
        """Pre-process, such as pre-pad and mod pad, so that the images can be divisible
        """
        img = torch.from_numpy(np.transpose(img, (2, 0, 1))).float()
        self.img = img.unsqueeze(0).to(self.device)
        if self.half:
            self.img = self.img.half()

        # pre_pad
        if self.pre_pad != 0:
            self.img = F.pad(self.img, (0, self.pre_pad, 0, self.pre_pad), 'reflect')
        # mod pad for divisible borders
        if self.scale == 2:
            self.mod_scale = 2
        elif self.scale == 1:
            self.mod_scale = 4
        if self.mod_scale is not None:
            self.mod_pad_h, self.mod_pad_w = 0, 0
            _, _, h, w = self.img.size()
            if (h % self.mod_scale != 0):
                self.mod_pad_h = (self.mod_scale - h % self.mod_scale)
            if (w % self.mod_scale != 0):
                self.mod_pad_w = (self.mod_scale - w % self.mod_scale)
            self.img = F.pad(self.img, (0, self.mod_pad_w, 0, self.mod_pad_h), 'reflect')

    def process(self):
        # model inference
        self.output = self.model(self.img)

    def tile_process(self):
        """It will first crop input images to tiles, and then process each tile.
        Finally, all the processed tiles are merged into one images.

        Modified from: https://github.com/ata4/esrgan-launcher
        """
        batch, channel, height, width = self.img.shape
        output_height = height * self.scale
        output_width = width * self.scale
        output_shape = (batch, channel, output_height, output_width)

        # start with black image
        self.output = self.img.new_zeros(output_shape)
        tiles_x = math.ceil(width / self.tile_size)
        tiles_y = math.ceil(height / self.tile_size)

        # loop over all tiles
        for y in range(tiles_y):
            for x in range(tiles_x):
                # extract tile from input image
                ofs_x = x * self.tile_size
                ofs_y = y * self.tile_size
                # input tile area on total image
                input_start_x = ofs_x
                input_end_x = min(ofs_x + self.tile_size, width)
                input_start_y = ofs_y
                input_end_y = min(ofs_y + self.tile_size, height)

                # input tile area on total image with padding
                input_start_x_pad = max(input_start_x - self.tile_pad, 0)
                input_end_x_pad = min(input_end_x + self.tile_pad, width)
                input_start_y_pad = max(input_start_y - self.tile_pad, 0)
                input_end_y_pad = min(input_end_y + self.tile_pad, height)

                # input tile dimensions
                input_tile_width = input_end_x - input_start_x
                input_tile_height = input_end_y - input_start_y
                tile_idx = y * tiles_x + x + 1
                input_tile = self.img[:, :, input_start_y_pad:input_end_y_pad, input_start_x_pad:input_end_x_pad]

                # upscale tile
                try:
                    with torch.no_grad():
                        output_tile = self.model(input_tile)
                except RuntimeError as error:
                    print('Error', error)
                print(f'\tTile {tile_idx}/{tiles_x * tiles_y}')

                # output tile area on total image
                output_start_x = input_start_x * self.scale
                output_end_x = input_end_x * self.scale
                output_start_y = input_start_y * self.scale
                output_end_y = input_end_y * self.scale

                # output tile area without padding
                output_start_x_tile = (input_start_x - input_start_x_pad) * self.scale
                output_end_x_tile = output_start_x_tile + input_tile_width * self.scale
                output_start_y_tile = (input_start_y - input_start_y_pad) * self.scale
                output_end_y_tile = output_start_y_tile + input_tile_height * self.scale

                # put tile into output image
                self.output[:, :, output_start_y:output_end_y,
                            output_start_x:output_end_x] = output_tile[:, :, output_start_y_tile:output_end_y_tile,
                                                                       output_start_x_tile:output_end_x_tile]

    def post_process(self):
        # remove extra pad
        if self.mod_scale is not None:
            _, _, h, w = self.output.size()
            self.output = self.output[:, :, 0:h - self.mod_pad_h * self.scale, 0:w - self.mod_pad_w * self.scale]
        # remove prepad
        if self.pre_pad != 0:
            _, _, h, w = self.output.size()
            self.output = self.output[:, :, 0:h - self.pre_pad * self.scale, 0:w - self.pre_pad * self.scale]
        return self.output

    @torch.no_grad()
    def enhance(self, img, outscale=None, alpha_upsampler='realesrgan'):
        h_input, w_input = img.shape[0:2]
        # img: numpy
        img = img.astype(np.float32)
        if np.max(img) > 256:  # 16-bit image
            max_range = 65535
            print('\tInput is a 16-bit image')
        else:
            max_range = 255
        img = img / max_range
        if len(img.shape) == 2:  # gray image
            img_mode = 'L'
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        elif img.shape[2] == 4:  # RGBA image with alpha channel
            img_mode = 'RGBA'
            alpha = img[:, :, 3]
            img = img[:, :, 0:3]
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if alpha_upsampler == 'realesrgan':
                alpha = cv2.cvtColor(alpha, cv2.COLOR_GRAY2RGB)
        else:
            img_mode = 'RGB'
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # ------------------- process image (without the alpha channel) ------------------- #
        self.pre_process(img)
        if self.tile_size > 0:
            self.tile_process()
        else:
            self.process()
        output_img = self.post_process()
        output_img = output_img.data.squeeze().float().cpu().clamp_(0, 1).numpy()
        output_img = np.transpose(output_img[[2, 1, 0], :, :], (1, 2, 0))
        if img_mode == 'L':
            output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2GRAY)

        # ------------------- process the alpha channel if necessary ------------------- #
        if img_mode == 'RGBA':
            if alpha_upsampler == 'realesrgan':
                self.pre_process(alpha)
                if self.tile_size > 0:
                    self.tile_process()
                else:
                    self.process()
                output_alpha = self.post_process()
                output_alpha = output_alpha.data.squeeze().float().cpu().clamp_(0, 1).numpy()
                output_alpha = np.transpose(output_alpha[[2, 1, 0], :, :], (1, 2, 0))
                output_alpha = cv2.cvtColor(output_alpha, cv2.COLOR_BGR2GRAY)
            else:  # use the cv2 resize for alpha channel
                h, w = alpha.shape[0:2]
                output_alpha = cv2.resize(alpha, (w * self.scale, h * self.scale), interpolation=cv2.INTER_LINEAR)

            # merge the alpha channel
            output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2BGRA)
            output_img[:, :, 3] = output_alpha

        # ------------------------------ return ------------------------------ #
        if max_range == 65535:  # 16-bit image
            output = (output_img * 65535.0).round().astype(np.uint16)
        else:
            output = (output_img * 255.0).round().astype(np.uint8)

        if outscale is not None and outscale != float(self.scale):
            output = cv2.resize(
                output, (
                    int(w_input * outscale),
                    int(h_input * outscale),
                ), interpolation=cv2.INTER_LANCZOS4)

        return output, img_mode


class PrefetchReader(threading.Thread):
    """Prefetch images.

    Args:
        img_list (list[str]): A image list of image paths to be read.
        num_prefetch_queue (int): Number of prefetch queue.
    """

    def __init__(self, img_list, num_prefetch_queue):
        super().__init__()
        self.que = queue.Queue(num_prefetch_queue)
        self.img_list = img_list

    def run(self):
        for img_path in self.img_list:
            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            self.que.put(img)

        self.que.put(None)

    def __next__(self):
        next_item = self.que.get()
        if next_item is None:
            raise StopIteration
        return next_item

    def __iter__(self):
        return self


class IOConsumer(threading.Thread):

    def __init__(self, opt, que, qid):
        super().__init__()
        self._queue = que
        self.qid = qid
        self.opt = opt

    def run(self):
        while True:
            msg = self._queue.get()
            if isinstance(msg, str) and msg == 'quit':
                break

            output = msg['output']
            save_path = msg['save_path']
            cv2.imwrite(save_path, output)
        print(f'IO worker {self.qid} is done.')


def FileExtension(FileName):
  return FileName.split(".")[-1]

#@title Main Code
# And this is the main method to use to enhance our images:
def enhance_image(input_file, layers=2, upscale=2, final_filename="", enhance_faces=False):
  ## Set models to use
  if layers == 4:
    # 4 layers
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
    netscale = 4
    model_file = 'RealESRGAN_x4plus.pth'
  elif layers == 2:
    # 2 layers
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
    netscale = 2
    model_file = 'RealESRGAN_x2plus.pth'
  else:
    print("Layers parameter must be either 2 or 4.")
    return
  # Final enhanced image will be upscaled by this factor using LANCZOS4 resampling
  # Input image
  imgname, org_extension = input_file.split('.')
  image = cv2.imread(input_file)
  org_width, org_height = image.shape[:2]
  # Compute tile size
  if min(org_width, org_height) <= 800:
    tile_size = 0
    print(f"Small image so batching is not necessary.")
  else:
    tile_size = ceil(sqrt(min(org_width, org_height))) * 10
  if tile_size > 500:
    tile_size = 350
  print(f"Tile size being used: {tile_size}")
  # restorer
  upsampler = RealESRGANer(
      scale=netscale,
      model_path=model_file,
      model=model,
      tile=tile_size,
      tile_pad=2,
      half=False)
  # Use GFPGAN for face enhancement
  if enhance_faces:
    from gfpgan import GFPGANer
    face_enhancer = GFPGANer(
        model_path='GFPGANv1.4.pth',
        upscale=upscale,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=upsampler)
  img = cv2.imread(input_file, cv2.IMREAD_UNCHANGED)
  try:
    if enhance_faces:
      _, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=False, paste_back=True)
    else:
      output, _ = upsampler.enhance(img, outscale=upscale)
  except RuntimeError as error:
      print('Error', error)
      print('If you encounter CUDA out of memory, try to set --tile with a smaller number.')
      print('Else, the file you are using may be too large.')
  else:
    if final_filename != "":
      save_path = final_filename
    else:
      save_path = f'{imgname}_out.{FileExtension(input_file)}'
    cv2.imwrite(save_path, output)
    print(f"Enhanced image has been saved to {save_path}.\nClick refresh button on the left panel to get latest version of {save_path}")
    return save_path




#@title Metrics Code
# These are some helper fucntions to compute the quality of images based on certain metrics like resolution, sharpness, contrast, and noise:
def get_resolution(image):
    return image.shape[:2]

def get_noise_level(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute the Discrete Fourier Transform (DFT)
    f_transform = np.fft.fft2(gray)
    f_transform_shifted = np.fft.fftshift(f_transform)

    # Compute the magnitude spectrum
    magnitude_spectrum = np.abs(f_transform_shifted)

    # Calculate the noise level using the standard deviation of the magnitude spectrum
    noise_level = np.std(np.log1p(magnitude_spectrum))

    return round(noise_level, 2)

def get_sharpness(image):
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # Apply an edge-enhancing filter (Laplacian) and compute variance as a measure of sharpness
    laplacian = cv2.Laplacian(cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2GRAY), cv2.CV_64F)
    return round(laplacian.var(), 2)

def get_contrast(image):
    # Using Michelson contrast measure
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    I_max = np.max(gray)
    I_min = np.min(gray)
    print(I_min, I_max)

    contrast = (I_max - I_min) / (I_max + I_min)
    return round(contrast, 5)


def get_filesize(image_file):
  file_size = os.path.getsize(image_file)
  return round(file_size / 1_000_000, 2)


import time
class Timer:
    def __init__(self) -> None:
        self.start = 0
        self.end = 0

    def start(self):
        self.start = time.time()

    def end(self):
        self.end = time.time()
        elapsed_time = self.end - self.start
        print(f"Elapsed Time: {elapsed_time} seconds")


def print_quality(image_file):
    # Load the image
    image = cv2.imread(image_file)

    # Get image metrics
    resolution = get_resolution(image)
    noise_level = get_noise_level(image)
    sharpness = get_sharpness(image)
    try:
      contrast = get_contrast(image)
    except:
      contrast = "unknown"
    image_size = get_filesize(image_file)

    # Output the results
    print(f"Resolution: {resolution} pixels")
    print(f"Noise Level: {noise_level} dB")
    print(f"Sharpness: {sharpness}")
    print(f"Contrast: {contrast}")
    print(f"Size of image: {image_size} MB")

In [ ]:
#@title Here we go woohoo :)
#@markdown Ok so now you can run the code cell below and scroll a little down to below the code cell. It will prompt you to upload an image file. After it is uploaded, it will enhance your image and display it for you.
#@markdown
#@markdown You can change the values of "layers" and "enhance_faces" in the actual method below which has been cordoned off for you! ;)

MergeSessionsPath = "/content/drive/MyDrive/UpscaleVideo/MergeSessions"
OriginalOutputPath = "/content/OriginalOutput"
UpscaledOutputPath = "/content/drive/MyDrive/UpscaleVideo/UpscaledOutput"



%cd /content
Model = 'RealESRGAN_x2plus.pth (default) (For General Images)' # @param ["RealESRGAN_x2plus.pth (default) (For General Images)", "RealESRGAN_x4plus.pth (For General Images)", "RealESRNet_x4plus.pth (X4 model with MSE loss (over-smooth effects))", "ESRGAN_SRx4_DF2KOST_official-ff704c30.pth (official ESRGAN model)", "RealESRGAN_x4plus_netD.pth (fine-tuning)", "RealESRGAN_x2plus_netD.pth (fine-tuning)", "RealESRGAN_x4plus_anime_6B.pth (for anime images)", "RealESRGAN_x4plus_anime_6B_netD.pth (fine-tuning anime images)", "realesr-animevideov3.pth (For Animation Videos)"]
#@markdown Choose either 2 or 4 as the value here.
layers = 2 # @param [2, 4]
#@markdown This value indicates the number of times the output image's resolution needs to enlarged from the original.
upscale = 2 # @param {type:"number"}
#@markdown I honestly don't know what this does. enhance the faces of the characters?
enhance_faces=True # @param {type:"boolean"}
Display_Images=True # @param {type:"boolean"}

#@markdown This says how many images to transfer to other sessions at once.
Queue = 10 # @param {type:"integer"}

from google.colab import files
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import shutil

global images

def RemoveNumbersFromString(string):
  returnString = ""
  for char in string:
    if(not char.isnumeric()):
      returnString = returnString + char
  return returnString

def DeleteDriveRootNonsense():
  global images
  DriveRootList = listdir2("/content/drive/MyDrive")
  DeleteList = [f".{FileExtension(images[0])}",f" ().{FileExtension(images[0])}",f"_upscaled.{FileExtension(images[0])}",f"_upscaled ().{FileExtension(images[0])}","SessionNamesToDo","SessionNamesToDo ()","SessionTaken ().txt","SessionTaken.txt","ToProcess","ToProcess ()","MergeSessions","MergeSessions ()"]
  for item in DriveRootList:
    if(RemoveNumbersFromString(item) in DeleteList):
      if(os.path.isdir(f"/content/drive/MyDrive/{item}")):
        shutil.rmtree(f"/content/drive/MyDrive/{item}")
      if(os.path.isfile(f"/content/drive/MyDrive/{item}")):
        os.remove(f"/content/drive/MyDrive/{item}")

def DisplayImage(imagePath):
  img = mpimg.imread(imagePath)
  plt.imshow(img)
  plt.axis('off')
  plt.show()

def enhanceImage2(filename, outputFilename, layers, upscale, enhance_faces):
  # Start timing
  t = Timer()
  t.start
  ########################################################################################################################################################
  result = enhance_image(input_file=filename, layers=layers, upscale=upscale, final_filename=outputFilename, enhance_faces=enhance_faces)
  ########################################################################################################################################################
  print(f"result: {result}")
  t.end # End timing
  ending = result.split(".")[1]
  output = outputFilename
  if(Display_Images):
    print(filename)
    DisplayImage(filename)
    print(output)
    DisplayImage(output)

def NumberOfImage(ImageName):
  returnNumber = ""
  for x in ImageName:
    if(x.isnumeric()):
     returnNumber = returnNumber + x
  return int(returnNumber)

def GetImageOn():
  imagesDone = os.listdir(UpscaledOutputPath)
  imagesDone.sort()
  returnNumber = 0
  x = 1
  while(x < len(imagesDone) -1):
    if(NumberOfImage(imagesDone[x]) < NumberOfImage(imagesDone[x+1])-1):
      returnNumber = NumberOfImage(imagesDone[x])
      break
    x = x + 1
  return returnNumber

def listdir2(directoryToList):
  returnList = []
  if(os.path.exists(directoryToList)):
    listOfDir = os.listdir(directoryToList)
    listOfDir.sort()
    returnList = []
    for thing in listOfDir:
      if(os.path.isfile(f"{directoryToList}/{thing}")):
        returnList.append(thing)
      if(os.path.isdir(f"{directoryToList}/{thing}")):
        if(not thing == ".ipynb_checkpoints"):
          returnList.append(thing)
        else:
          !rm -r {directoryToList}/{thing}
  return returnList

def ImageExistsElsewhere(ImageName):
  returnValue = False
  global images
  sessionsAvailable = listdir2(f"{MergeSessionsPath}/ToProcess")
  NumberOfAssistSessionsAvailable = len(listdir2(f"{MergeSessionsPath}/ToProcess"))
  if(NumberOfAssistSessionsAvailable > 0):
    for session in sessionsAvailable:
      if(os.path.exists(f"{MergeSessionsPath}/ToProcess/{session}/{ImageName}")):
        returnValue = True
  if(os.path.exists(f"{UpscaledOutputPath}/{ImageName.replace('.','_upscaled.')}")):
    returnValue = True
  return returnValue

def ProgressBar(number_on,final,units=100):
  percent = round((round(number_on)/round(final))*100,2)
  returnText = f"{percent}% : ["
  new_number_on = round((units*(number_on/final)))
  for x in range(new_number_on):
    returnText = returnText + "█"
  for x in range(units-new_number_on):
    returnText = returnText + "_"
  returnText = returnText + "]"
  return returnText

def setup():
  session = int(open("/content/sessionNumber.txt","r").read())
  if(session == 1):
    if(not os.path.exists(f"{MergeSessionsPath}/setup")):
      os.makedirs(f"{MergeSessionsPath}/setup")
    if(not os.path.exists(f"{MergeSessionsPath}/MasterSessionTaken.txt")):
      print()
      print()
      print("MergeSessionsPath/setup contains this:")
      print()
      print(os.listdir(f"{MergeSessionsPath}"))
      f = open(f"{MergeSessionsPath}/MasterSessionTaken.txt", "w")
      f.close()
      print(f"SESSION IS: {session}, AKA the master session.")
      print()
      print()
      print("After establishing session, MergeSessionsPath contains this:")
      print()
      print(os.listdir(f"{MergeSessionsPath}"))
  else:
    if(not os.path.exists(f"{MergeSessionsPath}/setup/Session{session}Taken.txt")):
      x = 2
      while(os.path.exists(f"{MergeSessionsPath}/setup/Session{x}Taken.txt")):
        x = x + 1
      print()
      print()
      print("MergeSessionsPath/setup contains this:")
      print()
      print(os.listdir(f"{MergeSessionsPath}/setup"))
      f = open(f"{MergeSessionsPath}/setup/Session{x}Taken.txt", "w")
      f.close()
      session = x
      open("/content/sessionNumber.txt","w").write(str(session))
      print(f"SESSION IS {session} AKA an assist session.")
      print()
      print()
      print("After establishing session, MergeSessionsPath/setup contains this:")
      print()
      print(os.listdir(f"{MergeSessionsPath}/setup"))


def main():
  setup()
  %cd /content
  if(Model == 'RealESRGAN_x2plus.pth (default) (For General Images)' and (not os.path.exists("/content/RealESRGAN_x2plus.pth"))):
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth
  if(Model == 'RealESRGAN_x4plus.pth (For General Images)' and (not os.path.exists("//content/RealESRGAN_x4plus.pth"))):
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
  if(Model == 'RealESRNet_x4plus.pth (X4 model with MSE loss (over-smooth effects))' and (not os.path.exists("/RealESRNet_x4plus.pth"))):
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/RealESRNet_x4plus.pth
  if(Model == 'ESRGAN_SRx4_DF2KOST_official-ff704c30.pth (official ESRGAN model)' and (not os.path.exists("/ESRGAN_SRx4_DF2KOST_official-ff704c30.pth"))):
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/ESRGAN_SRx4_DF2KOST_official-ff704c30.pth
  if(Model == 'RealESRGAN_x4plus_netD.pth (fine-tuning)' and (not os.path.exists("/RealESRGAN_x4plus_netD.pth"))):
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.3/RealESRGAN_x4plus_netD.pth
  if(Model == 'RealESRGAN_x2plus_netD.pth (fine-tuning)' and (not os.path.exists("/RealESRGAN_x2plus_netD.pth"))):
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.3/RealESRGAN_x2plus_netD.pth
  if(Model == 'RealESRGAN_x4plus_anime_6B.pth (for anime images)' and (not os.path.exists("/RealESRGAN_x4plus_anime_6B.pth"))):
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth
  if(Model == 'RealESRGAN_x4plus_anime_6B_netD.pth (fine-tuning anime images)' and (not os.path.exists("/RealESRGAN_x4plus_anime_6B_netD.pth"))):
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B_netD.pth
  if(Model == 'realesr-animevideov3.pth (For Animation Videos)' and (not os.path.exists("/realesr-animevideov3.pth"))):
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth

  if(not os.path.exists("/content/GFPGANv1.4.pth")):
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth



  if(not os.path.exists(UpscaledOutputPath)):
    os.makedirs(UpscaledOutputPath)

  session = int(open("/content/sessionNumber.txt","r").read())
  if(session == 1):
    print("THIS IS THE MASTER SESSION.")
    global images
    images = os.listdir(OriginalOutputPath)
    images.sort()
    fullAmount = len(images)
    imageOn = GetImageOn()
    x = imageOn
    while(x < fullAmount):
      filename = images[x]
      if(not ImageExistsElsewhere(filename)):
        #Checks to see if there are any other sessions running that are linked.
        if(os.path.exists(f"{MergeSessionsPath}/ToProcess")):
          assistSessionsAvailable = listdir2(f"{MergeSessionsPath}/ToProcess")
          NumberOfAssistSessionsAvailable = len(listdir2(f"{MergeSessionsPath}/ToProcess"))
          #Checks to see if there are any other sessions running that are linked.
          if(NumberOfAssistSessionsAvailable > 0):
            #Goes through all of the colab sessions that are running.
            #NOTE: session2 does NOT mean the 2nd session. It means the name of the session folder.
            for session2 in assistSessionsAvailable:
              #Checks to see if the colab session is free, and has no images to process.
              if(len(listdir2(f"{MergeSessionsPath}/ToProcess/{session2}")) < 1):
                #Moves the next Queue images that need to be processed, to the free session to process
                #Note: This does not move the image itself. It just moves the NAME of the image. This speeds up the runtime.
                print(f"Moving {Queue} images to alternate session, named: {session2}")
                x2 = 0
                while(x2 < Queue):
                  filename = images[x]
                  if(not ImageExistsElsewhere(filename)):
                    print(filename)
                    f = open(f"{MergeSessionsPath}/ToProcess/{session2}/{filename}","w")
                    f.close()
                    x2 = x2 + 1
                  x = x + 1
        filename = images[x]
        if(not ImageExistsElsewhere(filename)):
          print(f"processing: {filename}")
          DeleteDriveRootNonsense()
          enhanceImage2(f"{OriginalOutputPath}/{filename}",f"{UpscaledOutputPath}/{filename.replace('.','_upscaled.')}",layers,upscale,enhance_faces)
          print(f"{ProgressBar(x,fullAmount,40)} {x}/{fullAmount}")
      x = x + 1
    print("FINISHED with main program.")
    print("Checking for missing frames, and processing them.")
    for image in images:
      if(not os.path.exists(f"{UpscaledOutputPath}/{image.replace('.','_upscaled.')}")):
        print(f"processing: {image}}")
        enhanceImage2(f"{OriginalOutputPath}/{image}",f"{UpscaledOutputPath}/{image.replace('.','_upscaled.')}",layers,upscale,enhance_faces)
  else:
    print(f"Session is {session}, an assist session")
    while(os.path.exists(f"{MergeSessionsPath}/MasterSessionTaken.txt")):
      if(not os.path.exists(f"{MergeSessionsPath}/ToProcess/Session{session}NamesToDo")):
        os.makedirs(f"{MergeSessionsPath}/ToProcess/Session{session}NamesToDo")
      print("\r",f"Waiting for folder: \"{MergeSessionsPath}/ToProcess/Session{session}NamesToDo\" to be filled with a queue.", end="")
      images = os.listdir(f"{MergeSessionsPath}/ToProcess/Session{session}NamesToDo")
      images.sort()
      if(len(images) > 0):
        print()
        print("INFORMATION: RECEIVED INPUT!!! Hooray!")
        for image2 in images:
          print(f"processing {image2}")
          DeleteDriveRootNonsense()
          enhanceImage2(f"{OriginalOutputPath}/{image2}",f"{UpscaledOutputPath}/{image2.replace('.','_upscaled.')}",layers,upscale,enhance_faces)
          !rm {MergeSessionsPath}/ToProcess/Session{session}NamesToDo/{image2}
    print("Detected Master session has been stopped. Deleting queue directory.")
    !rm -r {MergeSessionsPath}/ToProcess/Session{session}NamesToDo
    print("Deleting session taken file.")
    !rm {MergeSessionsPath}/setup/Session{session}Taken.txt



try:
  main()
except KeyboardInterrupt:
  print('Interrupted')
  session = int(open("/content/sessionNumber.txt","r").read())
  if(session > 1):
    !rm {MergeSessionsPath}/setup/Session{session}Taken.txt
    raise Exception("KeyboardInterrupt: You have terminated this colab notebook. Sending message to main notebook it is closed.")
  else:
    !rm {MergeSessionsPath}/MasterSessionTaken.txt
    !rm -r {MergeSessionsPath}/setup
    raise Exception("KeyboardInterrupt: You have terminated the main colab notebook. Terminating all others...")




In [ ]:
#@title Finalize by merging all frames into one video, and adding audio back.
import os
import cv2

MergeSessionsPath = "/content/drive/MyDrive/UpscaleVideo/MergeSessions"
OriginalOutputPath = "/content/OriginalOutput"
UpscaledOutputPath = "/content/drive/MyDrive/UpscaleVideo/UpscaledOutput"



def ProgressBar(number_on,final,units=100):
  percent = round((round(number_on)/round(final))*100,2)
  returnText = f"{percent}% : ["
  new_number_on = round((units*(number_on/final)))
  for x in range(new_number_on):
    returnText = returnText + "█"
  for x in range(units-new_number_on):
    returnText = returnText + "_"
  returnText = returnText + "]"
  return returnText



session = int(open("/content/sessionNumber.txt","r").read())
if(session == 1):
  print("reading saved framerate from the first cell.")
  FrameRate = int(open("/content/drive/MyDrive/UpscaleVideo/Framerate.txt","r").read())
  %cd /content/drive/MyDrive/UpscaleVideo/UpscaledOutput

  frameNames = os.listdir(UpscaledOutputPath) # Get list of file names
  frameNames.sort()
  print("Getting width and height.")
  #get width and height of first frame
  height, width, _ = cv2.imread(f"{UpscaledOutputPath}/{frameNames[0]}").shape
  #Start the thing.
  print("Combining frames into video. But no audio yet.")
  out = cv2.VideoWriter('/content/drive/MyDrive/UpscaleVideo/EnhancedFramesOnly.mp4', cv2.VideoWriter_fourcc(*'mp4v'), FrameRate, (width, height))
  x = 0
  lenframeNames = len(frameNames)
  while(x < lenframeNames):
    frameName = frameNames[x]
    print("\r",f"adding: {frameName} {ProgressBar(x,lenframeNames,50)} {x}/{lenframeNames}", end="")
    frame = cv2.imread(f"{UpscaledOutputPath}/{frameName}")
    out.write(frame)
    x = x + 1
  out.release()

  OriginalVideo = "/content/drive/MyDrive/UpscaleVideo/EnhancedFramesOnly.mp4"
  OriginalAudio = "/content/drive/MyDrive/UpscaleVideo/audio.wav"
  NewVideo = "/content/drive/MyDrive/UpscaleVideo/output.mp4"
  !rm {NewVideo}

  !pip install moviepy==1.0.3

  # Import everything needed to edit video clips
  from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

  # Load my video
  video = VideoFileClip(OriginalVideo)

  # Load my audio
  audio_clip = AudioFileClip(OriginalAudio)

  # Add the audio to the video
  final_video = video.set_audio(audio_clip)

  # Write the result to a file (many options available !)
  final_video.write_videofile(NewVideo)

In [ ]:
#@title Terminate session
Terminate_Session=False # @param {type:"boolean"}

if(Terminate_Session):
  print('Interrupted')
  session = int(open("/content/sessionNumber.txt","r").read())
  if(session > 1):
    !rm {MergeSessionsPath}/setup/Session{session}Taken.txt
    raise Exception("KeyboardInterrupt: You have terminated this colab notebook. Sending message to main notebook it is closed.")
  else:
    !rm {MergeSessionsPath}/MasterSessionTaken.txt
    !rm -r {MergeSessionsPath}/setup
    raise Exception("KeyboardInterrupt: You have terminated the main colab notebook. Terminating all others...")
